In [4]:
import pandas as pd 
import numpy as np
import csv


## to store training and testing data in data frames 
train_df = pd.read_csv(r"D:\Image-Recoginition-Activity2\train.csv")
test_df = pd.read_csv(r"D:\Image-Recoginition-Activity2\test.csv")

#print(train_df.head())

In [5]:
## Number of top most frequent landmark_ids
rank_number = 101

## Fraction of images from each landmark-id
sampling_rate = 0.02
random_state = 17 ## for reproducibility


## Groupby landmark ids & give count of these recurring ids
landmarks=train_df.groupby(by='landmark_id').count().loc[:,"id"]
#print(landmarks.head())
## sort the landmark in descending order 
l = landmarks.sort_values(ascending=False)
#print(l)

#l = landmarks.sort_values(by ="landmark_id",ascending=False)
#write_grp_by = landmarks.to_excel(r"C:\Users\Abhishek\Desktop\To_be_deleted.xlsx")
#print(l)


lmks = pd.concat([l,l/l.sum(),l.cumsum()/l.sum()],axis = 1 ,ignore_index=True)
lmks.columns = ['count','proportion','CumSum']
#print(lmks)

ranked = lmks[0:rank_number]
#print(ranked.index)

train_ordered = train_df[train_df.landmark_id.isin(ranked.index)]
#print(train_ordered)
## sAMPLING FROM EACH LANDMARK_ID ............................................

sample_gby = train_ordered.groupby(by = "landmark_id").apply(lambda x: x.sample(frac=sampling_rate, random_state=random_state))
#print(sample_gby)

sample_idx = sample_gby.index.levels[1]
train_sample = train_df.iloc[sample_idx, :]
#print(train_sample)



#print(train_sample.shape[0])
train_sample.to_csv(r"C:\Users\Abhishek\Desktop\keras_try\train_sample.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

train_sample_df = pd.read_csv(r"C:\Users\Abhishek\Desktop\keras_try\train_sample.csv")
#print(train_sample_df)



In [6]:
#print(train_sample['landmark_id'].value_counts()[:10])
#print(train_sample.shape[0])
landmark_dist = train_sample['landmark_id'].value_counts() / train_sample.shape[0]
print(landmark_dist.head())
#print(len(landmark_dist))
import matplotlib.pyplot as plt

plt.figure(figsize = (12,5))
fig = plt.bar(range(101),landmark_dist)
plt.xlabel("landmark")
plt.ylabel("Proportion of landmark in training set")


9633    0.115288
6051    0.115078
None    0.067409
6599    0.053339
9779    0.041999
Name: landmark_id, dtype: float64


Text(0, 0.5, 'Proportion of landmark in training set')

In [29]:
## Create train, Validation & test dataset 
from sklearn.model_selection import StratifiedShuffleSplit
#print(train_sample_df)
X = train_sample_df['id']
Y = train_sample_df['landmark_id']
#print(X,Y)
ss = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2)

for train_id,test_id in ss.split(X,Y):
    #print(X,Y)
    # Y_train stores landmark_id , X_train stores id
    
    X_train,X_temp = X.iloc[train_id],X.iloc[test_id]
    Y_train , Y_temp = Y.iloc[train_id],Y.iloc[test_id]

## X_train & Y_train gets reduced train set 
train_sample_ref = train_sample_df.loc[train_sample_df["id"].isin(X_train)]
#print(train_sample_ref["landmark_id"])
#refined_train_sample_df = train_sample_ref.to_csv(r"C:\Users\Abhishek\Desktop\keras_try\refined_train_set.csv") 
## X_temp & Y_temp gets the Testing set 
test_sample = train_sample_df.loc[train_sample_df["id"].isin(X_temp)]
#test_sample_df =  test_sample.to_csv(r"C:\Users\Abhishek\Desktop\keras_try\test_sample.csv")

## Seggregating  Validation samples 
ss1 =  StratifiedShuffleSplit(n_splits = 1, test_size = 0.5)
for train_id , test_id in ss1.split(X_temp,Y_temp):
    X_valid, X_test = X_temp.iloc[train_id], X_temp.iloc[test_id]
    Y_valid, Y_test = Y_temp.iloc[train_id], Y_temp.iloc[test_id]
validation_sample = train_sample_df.loc[train_sample_df["id"].isin(X_valid)]
#validation_sample_df =  validation_sample.to_csv(r"C:\Users\Abhishek\Desktop\keras_try\Validation_sample.csv")
#print(len(validation_sample["id"]))

TypeError: You have to supply one of 'by' and 'level'

In [28]:
print("Number of training images:", len(train_sample_ref))
print("Number of Test images:", len(test_sample))
print("Number of Validation images:", len(validation_sample))


Number of training images: 3809
Number of Test images: 953
Number of Validation images: 476


In [9]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense

input_shape = (128,128,3)
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size =(4,4), activation='relu',input_shape = input_shape))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128, kernel_size=(2,2), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=256, kernel_size=(2,2), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='softmax'))

model.summary()

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])                     

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 125, 125, 16)      784       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxP

In [82]:

import os
import copy
from keras.preprocessing import image                  
from keras.preprocessing.image import ImageDataGenerator
import re
from keras.utils import to_categorical

def get_label_images(path,sample):
    sample_match = path.split('\\')
   
    counter =0
    if "Train_images" in sample_match:
        type_of_dataset = "Train_images"
    else:
        type_of_dataset = "Validation_images"
    all_files = os.listdir(path)
    full_file_path = [path+"\\"+each_file for each_file in all_files ]
    full_file_path =[each_file.replace('\\','/') for each_file in full_file_path]
    #print("full_file_path",len(full_file_path))
    sample_dup_file_paths = copy.deepcopy(full_file_path)
    label_out = pd.Series(name = "result")
    for ind,each in enumerate(full_file_path):
        match = re.search("(?<={0}/)\S+(?=.jpg)".format(type_of_dataset),each)
        #print(match.group())
        uniq_id = sample[sample["id"]==match.group()]["landmark_id"]
        #print(uniq_id)
        if uniq_id.empty ==True:
            sample_dup_file_paths.remove(each)

            
        label_out = label_out.append(uniq_id)
    
    label_out = np.array(pd.get_dummies(label_out))
    label_out = (label_out.tolist())

    dict_processed = pd.DataFrame({"full file path":sample_dup_file_paths,"label":label_out})
    #blah = dict_processed.to_csv(r"C:\Users\Abhishek\Desktop\keras_try\verify.csv")

    return dict_processed

#clas_name = pd.Series(["class"])
train_path = r"C:\Users\Abhishek\Desktop\keras_try\Train_images"
train_df = get_label_images(train_path,train_sample_ref)
#print(train_df.head())
train_class_list = []
for each_list in train_df["label"]:
    for ind,each_value in enumerate(each_list):
        if each_value == 1:
            train_class_list.append(str(ind))
           



train_df["class"]=train_class_list
#print(train_df.head())

train_datagen = ImageDataGenerator(rescale = 1/255)
train_pixel_dat_gen=train_datagen.flow_from_dataframe(train_df, directory=None, x_col='full file path', y_col = "class",target_size=(128,128), color_mode='rgb',class_mode='categorical', batch_size=100)





Found 2658 images belonging to 100 classes.


In [87]:
validation_path = r"C:\Users\Abhishek\Desktop\keras_try\Validation_images"

all_files = os.listdir(validation_path)
full_file_path = [validation_path+"\\"+each_file for each_file in all_files ]
full_file_path =[each_file.replace('\\','/') for each_file in full_file_path]
print("full_file_path",len(full_file_path))


# df_validation = pd.read_csv(r"C:\Users\Abhishek\Desktop\keras_try\Validation_sample.csv")
# print(df_validation.head())

# valid_labels = []

# for ind,each in enumerate(full_file_path):
#     match = re.search("(?<={0}/)\S+(?=.jpg)".format("Validation_images"),each)
#     #print(match.group())
#     uniq_id = df_validation[df_validation["id"]==match.group()]["landmark_id"]
#     #print(uniq_id)
#     valid_labels.append(uniq_id)
    
# validation_tuple = tuple((full_file_path,valid_labels))
# print(validation_tuple[0])

#print(valid_df["label"].shape)

# valid_datagen = ImageDataGenerator(rescale=1/255)


# valid_class_list = []

# for each_list in valid_df["label"]:
#     for ind,each_value in enumerate(each_list):
#         if each_value == 1:
#             valid_class_list.append(str(ind))

# valid_df["class"] = valid_class_list


#valid_pixel_dat_gen = valid_datagen.flow_from_dataframe(valid_df,directory=None,x_col = "full file path",y_col = "class",target_size=(128,128),class_mode='categorical',color_mode='rgb',batch_size=100)

#hist = model.fit_generator(train_pixel_dat_gen,steps_per_epoch=18,epochs=18,validation_data=valid_pixel_dat_gen,validation_steps=8)



full_file_path 412
                 id                                                url  \
0  f1710cb352188505  https://lh5.googleusercontent.com/-040Z3oFuD9g...   
1  7f73e3e715025677  https://lh4.googleusercontent.com/-vMOseSXpyvY...   
2  132d030552248853  https://lh6.googleusercontent.com/-yfnppXYYNFM...   
3  75ca19ea94fa51b6  http://lh5.ggpht.com/-65wFlhtCFzU/Ru2J2mPl0FI/...   
4  21fa91eba2d927fd  http://lh4.ggpht.com/-oVdDDu5vmmw/S_Ar-WMOT2I/...   

  landmark_id  
0        6051  
1        6599  
2        2061  
3        9633  
4        9633  
['C:/Users/Abhishek/Desktop/keras_try/Validation_images/003b6c48bbf39be9.jpg', 'C:/Users/Abhishek/Desktop/keras_try/Validation_images/0128aa8c1cc6c6a7.jpg', 'C:/Users/Abhishek/Desktop/keras_try/Validation_images/014a20157ddf5402.jpg', 'C:/Users/Abhishek/Desktop/keras_try/Validation_images/02d511063ad3dfed.jpg', 'C:/Users/Abhishek/Desktop/keras_try/Validation_images/02fd213264a9f329.jpg', 'C:/Users/Abhishek/Desktop/keras_try/Validation_i

In [88]:
## Model Training 

from keras.callbacks import ModelCheckpoint

epochs = 10

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_baseCNN.hdf5', 
                               verbose=1, save_best_only=True)

hist = model.fit_generator(train_pixel_dat_gen, steps_per_epoch=10,epochs=10,validation_data=validation_tuple,validation_steps=None)

AttributeError: 'str' object has no attribute 'ndim'